In [7]:
# USEFUL FUNCTIONS

# results = sp.search(q='urbancoyotes', limit=20)
# for idx, track in enumerate(results['tracks']['items']):
#    print(idx, track['name'])
    
# import json
# playlist_id = 'spotify:user:spotifycharts:playlist:37i9dQZEVXbJiZcmkrIHGU'
# results = sp.playlist(playlist_id)
# print(json.dumps(results, indent=4))

# birdy = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
# results = spotify.artist_albums(birdy, album_type='album')
# albums = results['items']
# while results['next']:
#     results = spotify.next(results)
#     albums.extend(results['items'])

# for album in albums:
#     print(album['name'])

# artist = sp.artist(birdy)
# print(artist)

# print("")
# user = sp.user('kolsen118')
# print(user)

In [100]:
# We used Spotipy's documentation, it is very useful!
# https://spotipy.readthedocs.io/en/2.12.0/

In [38]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [5]:
%env SPOTIPY_CLIENT_ID = 3543bc68b7a64fa4896cb0c1d2499e33
%env SPOTIPY_CLIENT_SECRET = b711f5fe2e9a417c89247f7036c9c069

env: SPOTIPY_CLIENT_ID=3543bc68b7a64fa4896cb0c1d2499e33
env: SPOTIPY_CLIENT_SECRET=b711f5fe2e9a417c89247f7036c9c069


In [110]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from collections import Counter

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [108]:
#prints number of playlists with corona_terms in title/description
#this is helpful bc it helps us see ahead of time how huge the dataset is
corona_terms = ["Quarantine"]
for term in corona_terms:
    playlists = sp.search('q="{}"'.format(term), type='playlist')
number_playlists = playlists['playlists']['total']
print(number_playlists)

289213


In [109]:
#function that creates a list with the playlist name & total number of tracks in playlist
def extract_playlist(x,list_):
    for i in x['playlists']['items']:
        list_.append({"name" : i['name'], "total":i["tracks"]['total'], "uri":i["uri"]})

In [111]:
#create a list of playlists

list_of_playlists = []

for term in corona_terms:
    
    #use index (i) which are multiples of 50 -- random sample bc of search limitation
    #spotify can only search max 2000 playlists at a time
    for i in range(0,2000,50):
        
        try:
            #create variable init_data that has random sample of playlists with terms
            playlist_data = sp.search('q="{}"'.format(term), type='playlist', limit=50, offset=i)
                        #limit = 50, can only return up to 50 items
                        #type = playlist, only return playlists
                        #offset = i, the index of the first item to return
            extract_playlist(playlist_data,list_of_playlists)
            #above function appends init_data to list_of_playlists
        
        except:
            print("Error")

In [113]:
#creates dataframe with list of playlists, dropping duplicates & delete their index
playlists = pd.DataFrame(list_of_playlists).drop_duplicates().reset_index(drop=True)

In [114]:
#creates csv file from df
playlists.to_csv("playlists.csv")

In [116]:
#for a random sample of playlists, takes random sample of songs, creates a list of songs and their uris
tracks = []

#using each uri code listed in the playlist.csv
for uri in playlists['uri']:
        
        length = sp.playlist_tracks(uri)['total']
        #playlist_tracks(parameter) is playlist id, aka uri
        #length is the playlist length of each individual playlist
    
        for i in range(0,length, 50):
            #from 0 to playlist length, take every other 50th index
            
            playlist_data = sp.playlist_tracks(uri, limit=50, offset=i)
            #playlist_tracks get full details of playlists, input URI
            #playlist_data now has playlist track data
            #replacing old data
            
            try:
                for k in playlist_data['items']:
                    tracks.append(k['track']["uri"])
                    
            except:
                pass

In [117]:
len(tracks)

267878

In [118]:
#dataframe with individual track uri and count
df = pd.DataFrame.from_dict(Counter(songs), orient='index').reset_index().rename(columns={"index":"uri", 0:"count"})

In [119]:
df

,uri,count
0,spotify:track:2x59pa3xDf8hHZ4N3f5fGQ,1
1,spotify:track:2R6UrJ8uWbSIiHWmvRQvN8,1
2,spotify:track:4eJikwHYHJ3yMg1tvS1Mom,1
3,spotify:track:4SafR3a8senZ2r65hZWMKN,1
4,spotify:track:23W7gzqG81mz9cQyjGSP91,1
...,...,...
2303,spotify:track:2DexIlEkLb0hKRVtyoreXn,1
2304,spotify:track:7i6r9KotUPQg3ozKKgEPIN,1
2305,spotify:track:3D0bXrSv7O73vOaGOG8J9c,1
2306,spotify:track:2rBHnIxbhkMGLpqmsNX91M,1


In [120]:
#useful stats to help us understand frequency of the popular songs
#to know when we should cut off our data/if necessary
df[(df['count']>1) & (df['count']<900)].describe()

,count
count,180.000000
mean,2.627778
std,1.103729
min,2.000000
25%,2.000000
50%,2.000000
75%,3.000000
max,9.000000


In [121]:
#creates a csv file of the unfiltered song data
df.to_csv("song_list_unfiltered.csv")

In [122]:
#cuts off any tracks that appear less than twice (CAN CHANGE)
#this is useful bc we don't want songs that don't appear frequently
df_f = df[df['count']>2]

In [123]:
#data enrich, filters important variables, makes the dataframe usable
def data_enrich(x):
    data = sp.track(x['uri'])
    #sets data variable for each track
    
    name = data['name']
    pop = data['popularity']
    date = data['album']['release_date']
    explic = data['explicit']
    
    features = sp.audio_features(x['uri'])
    #uses spotify function to access audio features
    
    return {**x, 
            "name" : name, 
            'release_date': date, 
            "popularity" : pop,
            'explicit' : explic, 
            **features[0]}

In [124]:
final = df_f.apply(lambda x: data_enrich(x), axis=1, result_type='expand')

In [125]:
final

,uri,count,name,release_date,popularity,explicit,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
477,spotify:track:5dsYVdaWnWIRm419kNBbag,5,Can't Change Me,1999-01-01,45,False,0.480,0.829,7,-6.302,...,0.000000,0.0978,0.548,162.942,audio_features,5dsYVdaWnWIRm419kNBbag,https://api.spotify.com/v1/tracks/5dsYVdaWnWIR...,https://api.spotify.com/v1/audio-analysis/5dsY...,202440,3
479,spotify:track:66R5oBIbtBDJk0D0oMVcVN,3,Nearly Forgot My Broken Heart,2015-09-18,1,False,0.598,0.806,0,-5.442,...,0.000003,0.1600,0.588,119.536,audio_features,66R5oBIbtBDJk0D0oMVcVN,https://api.spotify.com/v1/tracks/66R5oBIbtBDJ...,https://api.spotify.com/v1/audio-analysis/66R5...,234427,4
482,spotify:track:3vBA9cISon26MlnDWdKSSQ,5,Fell On Black Days,1994-03-09,5,False,0.577,0.732,4,-5.731,...,0.000204,0.1330,0.523,101.704,audio_features,3vBA9cISon26MlnDWdKSSQ,https://api.spotify.com/v1/tracks/3vBA9cISon26...,https://api.spotify.com/v1/audio-analysis/3vBA...,283027,3
483,spotify:track:7v6fE4H5BBK5591OjJM4gz,5,Thank You - Recorded Live At Esplanade Arts & ...,2011-01-01,4,False,0.399,0.316,7,-5.681,...,0.000000,0.7280,0.112,78.028,audio_features,7v6fE4H5BBK5591OjJM4gz,https://api.spotify.com/v1/tracks/7v6fE4H5BBK5...,https://api.spotify.com/v1/audio-analysis/7v6f...,288187,4
489,spotify:track:35SiNV5YHBDsuiTtD0CIwc,5,Hunger Strike,1991-01-01,57,False,0.370,0.563,0,-10.842,...,0.000002,0.0852,0.303,137.695,audio_features,35SiNV5YHBDsuiTtD0CIwc,https://api.spotify.com/v1/tracks/35SiNV5YHBDs...,https://api.spotify.com/v1/audio-analysis/35Si...,243027,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,spotify:track:44OxoDksTnkbNXW9OxiHgg,4,Flower,2017-03-10,40,False,0.469,0.792,7,-5.358,...,0.023700,0.2510,0.351,100.802,audio_features,44OxoDksTnkbNXW9OxiHgg,https://api.spotify.com/v1/tracks/44OxoDksTnkb...,https://api.spotify.com/v1/audio-analysis/44Ox...,206223,4
1645,spotify:track:0tUELgOuOJ3KCsYMDDsNvD,4,Nothing Compares 2 U - Live At SiriusXM/2015,2018-11-16,61,False,0.434,0.327,0,-10.720,...,0.000002,0.6860,0.295,119.506,audio_features,0tUELgOuOJ3KCsYMDDsNvD,https://api.spotify.com/v1/tracks/0tUELgOuOJ3K...,https://api.spotify.com/v1/audio-analysis/0tUE...,303907,4
1656,spotify:track:4y15UP1z4IMjzyw1FZV3Km,4,Doesn't Remind Me - Recorded Live At Fitzgeral...,2011-01-01,39,False,0.383,0.531,4,-3.118,...,0.000000,0.6890,0.322,129.218,audio_features,4y15UP1z4IMjzyw1FZV3Km,https://api.spotify.com/v1/tracks/4y15UP1z4IMj...,https://api.spotify.com/v1/audio-analysis/4y15...,248253,4
1690,spotify:track:65wiJjPTfdsaUNg9DAGGll,3,Can't Change Me - Recorded Live At Queen Eliza...,2011-01-01,36,False,0.403,0.430,2,-4.203,...,0.000000,0.7460,0.342,102.081,audio_features,65wiJjPTfdsaUNg9DAGGll,https://api.spotify.com/v1/tracks/65wiJjPTfdsa...,https://api.spotify.com/v1/audio-analysis/65wi...,220173,4


In [126]:
#makes a csv with the final dataset
final.to_csv("song_list.csv",)